Los nuevos vídeos van en paralelo a la carretera. Vamos a ver una longitud X de la misma. Vamos a intentar coger X metros. 10 por ejemplo. Calculamos los segundos que tarda en recorrerlo(así calculamos la velocidad). Tamaño en pixeles de la forma del coche, a partir de un umbral X el vehiculo será pesado. 



In [1]:
#Suponiendo que ya hemos trozeado los vídeos con el notebook llamado procesado_recorte_video_formatoOriginal
# Por lo tanto, tendremos un vídeo de 163 segundos. Vamos a hacer un ejemplo de uno de los vídeos y generar un dataset 
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [2]:
cap = cv2.VideoCapture('/content/drive/MyDrive/INGESTION/DATOS_MEDIDAS/Medidas_Junio_Aoiz/video_220613_15_42_42.MOV')
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

In [3]:
cont=0
while(cap.isOpened()):
    cont=cont+1
    ret, frame = cap.read()
    #print(ret)
    if ret == False: break
    if cont>20:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Dibujamos un rectángulo en frame, para señalar el estado
        # del área en análisis (movimiento detectado o no detectado)
        cv2.rectangle(frame,(0,0),(frame.shape[1],40),(0,0,0),-1)
        color = (0, 255, 0)
        texto_estado = "Estado: No se ha detectado movimiento"
        
        #print(frame.shape[0],frame.shape[1])
        # Especificamos los puntos extremos del área a analizar
        area_pts = np.array([[1000,0], [frame.shape[1],0], [frame.shape[1],500], [1000,500]])

        # Con ayuda de una imagen auxiliar, determinamos el área sobre la cual actuará el detector de movimiento
        imAux = np.zeros(shape=(frame.shape[:2]), dtype=np.uint8)
        imAux = cv2.drawContours(imAux, [area_pts], -1, (255), -1)
        image_area = cv2.bitwise_and(gray, gray, mask=imAux)
        
        # Obtendremos la imagen binaria donde la región en blanco que representa la existencia de movimiento
        fgmask_1 = fgbg.apply(image_area)
        fgmask_1 = cv2.morphologyEx(fgmask_1, cv2.MORPH_OPEN, kernel)
        fgmask_1 = cv2.dilate(fgmask_1, None, iterations=2)
        
        # Encontramos los contornos presentes en fgmask, para luego basándonos
        # en su área poder determina si existe movimiento
        cnts_1 = cv2.findContours(fgmask_1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        
        if len(cnts_1)>0:
            for cnt in cnts_1:
                if cv2.contourArea(cnt) > 500  :
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(frame, (x,y), (x+w, y+h),(0,255,0), 2)
                    texto_estado = "Movimiento!"
                    color = (0, 0, 255)
            
            # Visuzalizamos el rectangulo alrededor del área que vamos a analizary el estado de la detección de movimiento 
            cv2.drawContours(frame, [area_pts], -1, color, 2)
            cv2.putText(frame, texto_estado , (10, 750), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2) 
            cv2_imshow(frame)
            cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

Con el codigo anterior ya derectamos el recorrido del coche en un area determinada. Viene a ser un rectangulo de aproximagamente 500x920 pìxeles. Hemos colocado el limite de área para un objeto detectado en 500 píxeles. Así evitamos la aparición de agentes externos como particulas o pajaros. 

Ahora solo falta saber a cuantos metros equivalen en la realidad el área abarcada. Contar el número de frames que tarda el coche en recorrer esa distancia y tendríamos la velocidad.

Umbrales preliminares viendo el vídeo:

  -Vehículos ligeros, varían entre 1700 y 2200 píxels cuadrados. Para cubrirnos, vamos a aumentar este umbral de 1500 a 2500.

  -Motocicletas:Todo lo que esté por encima de 500 y menor a 1500 será una moto.

  -Vehículos pesados: Todo lo que esté por encima de 2500

Para el dataset tambíen es interesante los segundos de vídeo que estamos viendo a este coche. De la misma forma que con la velocidad, sabiendo los frames que cubrimos podemos calcular facilmente los segundos de vídeo a los que corresponde.


In [4]:
frame=0
while(cap.isOpened()):
    frame = frame + 1
    ret, frame_video = cap.read()
    #print(ret)
    if ret == False: break
    if frame>20:
        gray = cv2.cvtColor(frame_video, cv2.COLOR_BGR2GRAY)
        # Dibujamos un rectángulo en frame, para señalar el estado
        # del área en análisis (movimiento detectado o no detectado)
        cv2.rectangle(frame_video,(0,0),(frame_video.shape[1],40),(0,0,0),-1)
        color = (0, 255, 0)
        texto_estado = "Estado: No se ha detectado movimiento"
        
        #print(frame.shape[0],frame.shape[1])
        # Especificamos los puntos extremos del área a analizar
        area_pts = np.array([[1000,0], [frame_video.shape[1],0], [frame_video.shape[1],500], [1000,500]])

        # Con ayuda de una imagen auxiliar, determinamos el área sobre la cual actuará el detector de movimiento
        imAux = np.zeros(shape=(frame_video.shape[:2]), dtype=np.uint8)
        imAux = cv2.drawContours(imAux, [area_pts], -1, (255), -1)
        image_area = cv2.bitwise_and(gray, gray, mask=imAux)
        
        # Obtendremos la imagen binaria donde la región en blanco que representa la existencia de movimiento
        fgmask_1 = fgbg.apply(image_area)
        fgmask_1 = cv2.morphologyEx(fgmask_1, cv2.MORPH_OPEN, kernel)
        fgmask_1 = cv2.dilate(fgmask_1, None, iterations=2)
        
        # Encontramos los contornos presentes en fgmask, para luego basándonos
        # en su área poder determina si existe movimiento
        cnts_1 = cv2.findContours(fgmask_1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        
        if len(cnts_1)>0:
            for cnt in cnts_1:
                if cv2.contourArea(cnt) > 500 and cv2.contourArea(cnt)<1500  :
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(frame_video, (x,y), (x+w, y+h),(0,255,0), 2)
                    texto_estado = "Movimiento moto!"
                    color = (0, 0, 255)
                    tipo_vehiculo = "moto"
                if cv2.contourArea(cnt) > 1500 and cv2.contourArea(cnt) < 2500  :
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(frame_video, (x,y), (x+w, y+h),(0,255,0), 2)
                    texto_estado = "Movimiento: coche!"
                    color = (0, 0, 255)
                    tipo_vehiculo = "coche"
                if cv2.contourArea(cnt) > 2500 :
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(frame_video, (x,y), (x+w, y+h),(0,255,0), 2)
                    texto_estado = "Movimiento: camión!"
                    color = (0, 0, 255)
                    tipo_vehiculo = "camión"
            
            # Visuzalizamos el rectangulo alrededor del área que vamos a analizary el estado de la detección de movimiento 
            cv2.drawContours(frame_video, [area_pts], -1, color, 2)
            cv2.putText(frame_video, texto_estado , (10, 750), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2) 
            cv2.imshow("frame",frame_video)
            cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

Ahora mismo no tenemos en cuenta si se cruzasen dos coches en el área determinada. Para tener esto en cuenta nos vamos a basar en el siguiente script que consigue el "tracking" de una persona pero aplicado para un coche. 
https://broutonlab.com/blog/opencv-object-tracking